In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import jlc
import scipy
import cv2
from utils import estimate_bias, estimate_bias_coefs,norm_quantile,norm_translation,process_vol,soft_threshold,get_disc_2d,inspect_fiber_vol
from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
filenames = list(Path("./data/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data\\YCsgA_Exp2_Pos1.tif', 'data\\YCsgA_Exp2_Pos2.tif', 'data\\YCsgA_Exp2_Pos3.tif', 'data\\YCsgA_Exp2_Pos4.tif', 'data\\YCsgA_Exp3_Pos1.tif', 'data\\YCsgA_Exp3_Pos2.tif', 'data\\YCsgA_Exp3_Pos3.tif', 'data\\YCsgA_Exp3_Pos4.tif']


In [5]:
vol = jlc.load_tifvol(filenames[0])
vol = vol[:,:,:vol.shape[2]//2].astype(float)
vol = norm_quantile(vol,alpha=0.001,clip=True)
coefs = estimate_bias_coefs(vol)
vol *= coefs.reshape(-1,1,1)
bias = estimate_bias(vol)
vol -= bias
vol = norm_quantile(vol,alpha=0.001,clip=True)
vol,best_translation = norm_translation(vol)

KeyboardInterrupt: 

In [4]:
images = []
for filename in filenames:
    vol = jlc.load_tifvol(filename.replace("data","data_processed"))
    images.append(vol[vol.shape[0]//2,:,:vol.shape[2]//2].astype(float))
jlc.montage(images)

In [10]:
jlc.montage([np.clip(im,0,1000) for im in images])

In [3]:
#process and save
for filename in filenames:
    savename = filename.replace("data","data_processed2")
    os.makedirs(os.path.dirname(savename),exist_ok=True)
    jlc.save_tifvol(process_vol(filename),savename)
    print("done with ",filename)

done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos1.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos2.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos3.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos4.tif
done

In [4]:
filenames = list(Path("./data_processed/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data_processed\\YCsgA_Exp2_Pos1.tif']


In [ ]:
#4,5,6,7: weird in the start
#2: one bad frame in start


In [7]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed"))
jlc.inspect_vol(vol.transpose(0,2,1))

In [ ]:
from utils import FiberGraph

fiber_graph = FiberGraph()
n = 10
for i in range(100):
    fiber_graph.add_node(np.random.rand()*n,np.random.rand()*n,i)


plt.imshow(np.zeros((n,n)),cmap="gray",extent=[0,n,0,n])
fiber_graph.plot()

In [28]:
from structure_tensor import eig_special_2d, structure_tensor_2d
image = vol[200]

sigma = 1
rho = 3
S = structure_tensor_2d(image, sigma, rho)
val, vec = eig_special_2d(S)

print(val.shape)
print(vec.shape)
print(S.shape)

(2, 684, 428)
(2, 684, 428)
(3, 684, 428)


In [29]:
plt.figure(figsize=(15, 15))
plt.subplot(221)
plt.imshow(image.transpose(1,0),cmap="gray")
plt.subplot(222)
val3 = np.concatenate((norm_quantile(val[0,:,:,None],alpha=0.01),
                       norm_quantile(val[1,:,:,None],alpha=0.01),
                       0*val[1,:,:,None]),axis=2)
plt.imshow(val3.transpose(1,0,2))
plt.subplot(223)
vec3 = np.concatenate((norm_quantile(vec[0,:,:,None],alpha=0.01),
                       norm_quantile(vec[1,:,:,None],alpha=0.01),
                       0*vec[1,:,:,None]),axis=2)
plt.imshow(vec3.transpose(1,0,2))
plt.subplot(224)
S3 = norm_quantile(S.transpose(2,1,0),alpha=0.01)
plt.imshow(S3)

In [105]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed"))
vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, 0.7, 0.7))
jlc.inspect_vol(soft_threshold(vol.transpose(0,2,1),0.1,0.2))

In [6]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed"))
vol = vol[:,200:300,200:350]
#vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, 1, 1))
#nereast
vol = scipy.ndimage.median_filter(vol, size=(9,1,1), mode='nearest')

#

In [26]:
# TODO
# Favor newer fibers when figuring out fiber origin
# filter out fibers that moved/dissapeard
# add calculation for fiber direction and include it in fiber origin selection
# dont let fibers do tiny branches

(100, 150)

In [102]:
from utils import FiberGraph
n = 100
fiber_graph = FiberGraph(radius=2)
"""for i in range(10):
    x,y = np.random.randint(n),np.random.randint(n)
    fiber_graph.add_node(x,y,i)
"""
mf = vol.shape[0]
fiber_graph.process_vol(vol,max_frames=mf)
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot()

In [101]:


inspect_fiber_vol(vol,fiber_graph=fiber_graph)

In [ ]:
jlc.inspect_vol()

In [62]:
plt.imshow(fiber_graph.mask_out_func())

In [12]:
jlc.inspect_vol(soft_threshold(vol,0.1,0.2))

In [41]:

plt.imshow(strict_local_max((vol[100]>0.15)*vol[100]),cmap="gray")

In [43]:
#find gaussian bell curve value at +1sigma:
gaussian = lambda x: np.exp(-x**2/2)
gaussian(1.2)/gaussian(0)

0.4867522559599717

In [36]:
plt.imshow((vol[100]>0.15)*vol[100])

In [34]:
jlc.inspect_vol(vol>0.15)

In [33]:
#structure = get_disc_2d(1)
i = 0
structure = np.ones((2,2))
plt.subplot(221)
plt.imshow(vol[i]>0.15)
closed = scipy.ndimage.binary_opening(vol[i]>0.15,structure=structure,iterations=1)
plt.subplot(222)
plt.imshow(closed)
plt.subplot(223)
plt.imshow(vol[i])

In [31]:
vol200 = np.concatenate([vol[200,:,:,None]]*3,2)
vol200[:,:,1][closed==True] *= 0
vol200[:,:,2][closed==True] *= 0 
plt.imshow(vol200*3)  

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
